In [1]:
!pip install -U wxPython

In [2]:
import string

import pandas as pd
import wx

In [3]:
def size_in_px(text, font_face="Arial", font_size=11, bold=False, italic=False):
    _ = wx.App()

    font_info = wx.FontInfo(font_size).FaceName(font_face)
    if bold:
        font_info = font_info.Bold()
    if italic:
        font_info = font_info.Italic()
    font = wx.Font(font_info)

    screen_dc = wx.ScreenDC()
    screen_dc.SetFont(font)
    size = screen_dc.GetTextExtent(text)

    return size

In [4]:
def prepare_df():
    from itertools import product

    FONT_FACES = [
        # main
        "Courier",
        "Geneva",
        "Georgia",
        "Helvetica",
        "Lucida Grande",
        "Times New Roman",
        "Verdana",
        # additional
        "Arial", # pretty standard
        "Brush Script MT", # very curved
        "Lucida Console", # monospaced
        "Wingdings", # just symbols
    ]
    FONT_SIZES = [
        9, 11, 12, 14, 17, 20,
    ]
    FONT_VERSIONS = [
        {'bold': False, 'italic': False},
        {'bold': True, 'italic': False},
        {'bold': False, 'italic': True},
        {'bold': True, 'italic': True},
    ]
    SYMBOLS = [(ord(c), c) for c in string.digits + string.punctuation + " "]
    BASIC_LATIN_CHARS = \
        [(i, chr(i)) for i in list(range(ord(u'\u0041'), ord(u'\u005a') + 1)) + \
                              list(range(ord(u'\u0061'), ord(u'\u007a') + 1)) \
         if chr(i).isprintable()] + SYMBOLS
    RUSSIAN_CHARS = \
        [(i, chr(i)) for i in range(ord(u'\u0410'), ord(u'\u044f') + 1) \
         if chr(i).isprintable()] + SYMBOLS
    GREEK_CHARS = \
        [(i, chr(i)) for i in list(range(ord(u'\u0391'), ord(u'\u03a9') + 1)) + \
                              list(range(ord(u'\u03b1'), ord(u'\u03c9') + 1)) \
         if chr(i).isprintable()]
    JAPANESE_HIRAGANA_CHARS = \
        [(i, chr(i)) for i in range(ord(u'\u3040'), ord(u'\u309f') + 1) \
         if chr(i).isprintable()]
    CHARS = \
        [(p[0], p[1], "basic_latin") for p in BASIC_LATIN_CHARS] + \
        [(p[0], p[1], "russian") for p in RUSSIAN_CHARS] + \
        [(p[0], p[1], "greek") for p in GREEK_CHARS] + \
        [(p[0], p[1], "japanese_hiragana") for p in JAPANESE_HIRAGANA_CHARS]

    data = {
        "char_id": [],
        "char": [],
        "alphabet": [],
        "font_face": [],
        "font_size": [],
        "font_version": [],
        "width": [],
    }

    for t in product(FONT_FACES, FONT_SIZES, FONT_VERSIONS, CHARS):
        data["font_face"] += [t[0]]
        data["font_size"] += [t[1]]
        data["font_version"] += [
            "bi" if t[2]['bold'] and t[2]['italic'] else
            "b" if t[2]['bold'] and not t[2]['italic'] else
            "i" if not t[2]['bold'] and t[2]['italic'] else ""
        ]
        data["char_id"] += [t[3][0]]
        data["char"] += [t[3][1]]
        data["alphabet"] += [t[3][2]]
        data["width"] += [size_in_px(t[3][1], t[0], t[1], t[2]['bold'], t[2]['italic'])[0]]

    return pd.DataFrame(data)

In [5]:
df = prepare_df()
df.head()

,char_id,char,alphabet,font_face,font_size,font_version,width
0,65,A,basic_latin,Courier,9,,9
1,66,B,basic_latin,Courier,9,,9
2,67,C,basic_latin,Courier,9,,9
3,68,D,basic_latin,Courier,9,,9
4,69,E,basic_latin,Courier,9,,9


In [6]:
df.to_csv("../data/char_widths.csv", index=False)